# Analyses

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
data=pd.read_csv('./lbc-export_20230611_1623.csv')
data.head()

,id,object,result_position,annonce_id,api_key,ad_type,area,category_name,currency,city,...,room_count,real_estate_type,title,url,urgent,user_id,collected_at,input_url,input_max_pages,reference
0,19590135,result,1,2363112865,54bb0281238b45a03f0ee695f73e704f,offer,108.0,Ventes immobilières,EUR,Sayat 63530,...,5.0,Maison,Maison 5 pièces,https://www.leboncoin.fr/ventes_immobilieres/2...,f,9f374671-c93b-4ad7-a437-ab815fa3cc72,2023-06-11 16:08:46.301605+00,https://www.leboncoin.fr/ventes_immobilieres/o...,99,NaN
1,19590137,result,2,2363113299,54bb0281238b45a03f0ee695f73e704f,offer,NaN,Ventes immobilières,EUR,Lyon 69002,...,1.0,Terrain,Terrain sur roujan 34320,https://www.leboncoin.fr/ventes_immobilieres/2...,f,82a473ef-43c5-4ad6-a7b8-435650010934,2023-06-11 16:08:46.941519+00,https://www.leboncoin.fr/ventes_immobilieres/o...,99,NaN
2,19590138,result,3,2363113246,54bb0281238b45a03f0ee695f73e704f,offer,NaN,Ventes immobilières,EUR,Allinges 74200 Noyer,...,1.0,Terrain,"Allinges, Terrain 480 m2, 150 000 euros",https://www.leboncoin.fr/ventes_immobilieres/2...,f,51f7583a-6022-449a-a7de-4c597474e5c0,2023-06-11 16:08:47.6019+00,https://www.leboncoin.fr/ventes_immobilieres/o...,99,NaN
3,19505587,result,4,2361918934,54bb0281238b45a03f0ee695f73e704f,offer,50.0,Ventes immobilières,EUR,Clermont-Ferrand 63100,...,3.0,Maison,Petite Maison plain-pied,https://www.leboncoin.fr/ventes_immobilieres/2...,f,79005959-5637-48f6-bfaf-17dd63e7c221,2023-06-11 16:08:48.29354+00,https://www.leboncoin.fr/ventes_immobilieres/o...,99,NaN
4,19590140,result,5,2363112836,54bb0281238b45a03f0ee695f73e704f,offer,200.0,Ventes immobilières,EUR,Divonne-les-Bains 01220,...,6.0,Maison,Maison de caractère rénovée - Divonne Les Bains,https://www.leboncoin.fr/ventes_immobilieres/2...,f,2e7dab91-4d01-43b8-b23b-25fa6601053d,2023-06-11 16:08:48.881353+00,https://www.leboncoin.fr/ventes_immobilieres/o...,99,NaN


In [16]:
data.describe()

,id,result_position,annonce_id,area,lat,lng,owner_type,owner_store_id,owner_siren,price,postal_code,room_count,input_max_pages
count,2.070000e+02,207.000000,2.070000e+02,190.000000,207.000000,207.000000,207.000000,2.070000e+02,1.830000e+02,2.070000e+02,207.000000,190.000000,207.0
mean,1.811533e+07,104.000000,2.317315e+09,142.631579,45.620745,4.959283,0.884058,3.286382e+07,6.528052e+08,3.391891e+05,48809.048309,4.800000,99.0
std,2.279718e+06,59.899917,5.620273e+07,144.113605,0.457582,1.059428,0.320932,2.177075e+07,2.083202e+08,3.000628e+05,25131.443188,2.739482,0.0
min,1.077151e+07,1.000000,2.026545e+09,9.000000,44.275400,2.353770,0.000000,5.111000e+03,3.015019e+08,4.300000e+01,1110.000000,1.000000,99.0
25%,1.683168e+07,52.500000,2.306022e+09,76.000000,45.312910,4.416680,1.000000,1.430110e+07,4.799726e+08,1.400000e+05,38050.000000,3.000000,99.0
50%,1.958965e+07,104.000000,2.336033e+09,110.500000,45.735450,4.889240,1.000000,3.288568e+07,6.057205e+08,2.350000e+05,63110.000000,5.000000,99.0
75%,1.959081e+07,155.500000,2.356793e+09,159.500000,45.931085,5.823340,1.000000,5.352192e+07,8.490126e+08,4.545000e+05,69600.000000,6.000000,99.0
max,1.959144e+07,207.000000,2.363113e+09,1445.000000,46.419820,6.797940,1.000000,6.868060e+07,9.625043e+08,2.205000e+06,74930.000000,20.000000,99.0


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 44 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      207 non-null    int64  
 1   object                  207 non-null    object 
 2   result_position         207 non-null    int64  
 3   annonce_id              207 non-null    int64  
 4   api_key                 207 non-null    object 
 5   ad_type                 207 non-null    object 
 6   area                    190 non-null    float64
 7   category_name           207 non-null    object 
 8   currency                207 non-null    object 
 9   city                    207 non-null    object 
 10  description             207 non-null    object 
 11  department              207 non-null    object 
 12  details                 207 non-null    object 
 13  DPE_string              207 non-null    object 
 14  DPE                     161 non-null    ob

In [23]:
cities = data.groupby('city')

In [25]:
data.loc[:, 'city'].head()

0                Sayat 63530
1                 Lyon 69002
2       Allinges 74200 Noyer
3     Clermont-Ferrand 63100
4    Divonne-les-Bains 01220
Name: city, dtype: object

In [29]:
data.city.value_counts()

city
Lyon 69002                      5
Lyon 69007 7e Arrondissement    4
Lyon 69008 8e Arrondissement    3
Givors 69700                    3
Reignier-Esery 74930            3
                               ..
Thoiry 01710                    1
Villeurbanne 69100              1
Neuville-sur-Ain 01160          1
Vinay 38470                     1
Caluire-et-Cuire 69300          1
Name: count, Length: 177, dtype: int64